Para poder utilizar IBMWatson primero te debes registrar en: 

https://www.ibm.com/cloud/watson-natural-language-understanding/faq



In [74]:
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, SentimentOptions, CategoriesOptions
import re
from unicodedata import normalize
import string

In [136]:
nlu = NaturalLanguageUnderstandingV1(                                         
    version='2018-11-16',
    iam_apikey='ME64j5wCJepkleMlF3zBCZGmF3riaERYqU8IX7WUD2Zq', 
    url='https://gateway-lon.watsonplatform.net/natural-language-understanding/api' 
    )

def analisis_sentimiento(input_text): 
    response = nlu.analyze (text = input_text,
    features = Features(sentiment=SentimentOptions()), language='es').get_result()
    # De la salida se obtiene un número entre -1 y 1
    res = response.get('sentiment').get('document').get('score')
    return res


C:\Users\Esperanza\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  after removing the cwd from sys.path.


In [76]:
# Es necesario quitar los acentos y las diéresis para no sustituir palabras como moño a mono (no podemos utilizar la libreia unicode)
#Se convierte todo el texto a minúsculas
#Se eliminan acentos y diéresis 
def estandarizacion(string):
    res=string.lower()
    res=re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", res), 0, re.I)
    res=normalize( 'NFC', res)
    return res


In [138]:
#Prueba con el dataset crea
### Preparación de la librería de insultos
# Obtenida de https://listas.20minutos.es/lista/insultos-en-castellano-que-deberias-conocer-y-su-significado-393340/
path_lib_insultos="C:/Users/Esperanza/Desktop/UOC/2año/TFM/libreria_insultos.txt"

with open(path_lib_insultos, encoding='utf8') as f:
    lines = f.read().splitlines()
    
#Se inicializa la lista
res_estand_lib_ins = []
#Se convierte en minúscula todos los elementos y se eliminan diéresis 
for i in lines:
    res_estand_lib_ins.append(estandarizacion(i))
    
#Convertimos a lista de frases la lista de palabras tóxicas 
sent_ibm=["Tú eres un " + strng for strng in res_estand_lib_ins]


res_pos=[]
res_neg=[]
for strng in sent_ibm: 
    aux=analisis_sentimiento(strng)
    if(aux>0):
        res_pos.append(strng)        
    else:
        res_neg.append(strng)

   

In [142]:
print("Total de palabras en la libería de insultos ", len(sent_ibm)) 
print("Porcentaje de palabras positivas ", len(res_pos)/len(sent_ibm))
print("Porcentaje de palabras negativas :", len(res_neg)/len(sent_ibm))  

Total de palabras en la libería de insultos  241
Porcentaje de palabras positivas  0.5850622406639004
Porcentaje de palabras negativas : 0.4149377593360996


In [87]:
analisis_sentimiento("Soy una mujer gay negra")

-0.647165

In [106]:
analisis_sentimiento("es que soy mujer!")

0.441942

In [105]:
analisis_sentimiento("Es que soy hombre!")

0.501246

In [112]:
analisis_sentimiento("eres imbecil :)")

0.727783

In [114]:
analisis_sentimiento("eres imbécil :)")

-0.897086

In [96]:
analisis_sentimiento("yo soy musulman")

-0.657952

In [97]:
analisis_sentimiento("yo soy cristiano")

-0.263908

In [115]:
analisis_sentimiento("Es que soy musulman")

-0.50646

In [116]:
analisis_sentimiento("Es que soy cristiano")

0